In [1]:
import sys
import xml.etree.ElementTree as ET


sys.path.append('../')
sys.path.append('../my_libs/')
sys.path.append('../my_libs/img/')

!{sys.executable} -m pip install --break-system-packages -r ../../pip-dependencies.txt

  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
   ---------------------------------------- 0.0/901.3 kB ? eta -:--:--
   --------- ------------------------------ 204.8/901.3 kB 4.1 MB/s eta 0:00:01
   ----------------------------- ---------- 665.6/901.3 kB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 901.3/901.3 kB 7.1 MB/s eta 0:00:00
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
Using cached multiprocess-0.70.16-py311-none-any.whl (143 kB)
   ---------------------------------------- 0.0/294.9 kB ? eta -:--:--
   ---------------------------------------- 294.9/294.9 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installati


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\jaros\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import my_libs as ml
from my_libs.tools import *
from my_libs.fft import *
from my_libs.colors import *
from my_libs.img.processing import *
from ultralytics import YOLO


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\jaros\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
ml.tools.printf("Hello world!")

Hello world!

## Cvičení 13(14)

0)  Použijte knihovnu ultralytics. 
Knihovnu lze nainstalovat jako pip install ultralytics 
1)  Převeďte data do formátu podporovaného ultralyics dle 
https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/  
Všechny anotace k obrázkům jsou v annotations.xml. Ke každému 
obrázku nazev.png v adresáři images je potřeba vytvořit odpovídající 
nazev.txt v adresáři labels. Každý objekt na obrázku je pak jeden 
řádek v txt souboru ve formátu 
class_idx x_center y_center width height 
kde  class_idx je index třídy a ostatní jsou souřadnice ohraničujícího 
obdélníka relativně vůči velikosti obrázku, viz 
https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#22-create-
labels  
2)  Důležitý je rovněž yaml soubor s cestami k datasetu a názvy tříd (v našem 
problému máme pouze jednu třídu „strawberry”), viz  
https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#21-create-
datasetyaml  
Data nemusíte rozdělovat na trénovací a validační, tzn. pro adresáře 
train i val použijte adresář obsahující všechny obrázky. 
3)  Stáhněte předtrénovaný YOLO model z 
https://docs.ultralytics.com/models/yolo11/#supported-tasks-and-modes 
dostatečně dobře bude fungovat i nejmenší model nano (n). 
4)  Natrénujte model a uložte, viz 
https://docs.ultralytics.com/modes/train/#usage-examples  
5)  Použijte model na obrázky v podadresáři test a vykreslete predikované 
obdélníky, viz 
https://docs.ultralytics.com/modes/predict/#key-features-of-predict-mode

Vykreslit results sám, ne pomocí zabudovaných funkcí

In [4]:
# Load a COCO-pretrained YOLO11n model
model = YOLO("yolo11n.pt")
#model2 = YOLO("yolo11n.pt")
#test_results = model2("datasets/strawberries/test/strawberry-bg-1300_800x.jpg")

""" for result in test_results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen """
#print(boxes)

' for result in test_results:\n    boxes = result.boxes  # Boxes object for bounding box outputs\n    masks = result.masks  # Masks object for segmentation masks outputs\n    keypoints = result.keypoints  # Keypoints object for pose outputs\n    probs = result.probs  # Probs object for classification outputs\n    obb = result.obb  # Oriented boxes object for OBB outputs\n    result.show()  # display to screen '

### Resize obrázky na 640x640px

In [5]:
input_folder = "strawberries/images"
output_folder = "strawberries/images_640px"

# NOT NEEDED IN THE END, DOES IT ITSELF
def resize_images(input_folder, output_folder):
    for file in os.listdir(input_folder):
        input_path = os.path.join(input_folder, file)
        output_path = os.path.join(output_folder, file)

        image = cv2.imread(input_path)
        image_640px = cv2.resize(image, (640, 640))

        cv2.imwrite(output_path, image_640px)
    print("Images resized and saved to: ", output_folder)

### Tvorba labels z annotations.xml

In [13]:
labels_folder = "datasets/strawberries/labels"
annot_folder = "datasets/strawberries/annotations.xml"

def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for image in root.findall("image"):
        txt_filename = labels_folder+"/"+image.attrib["id"]+".txt"
        txt_file = open(txt_filename, "w")
        image_width = int(image.attrib["width"])
        image_height = int(image.attrib["height"])
        for box in range(len(image)):
            # Relative 0-1 coords
            xtl = float(image[box].attrib["xtl"])/image_width # x top left
            ytl = float(image[box].attrib["ytl"])/image_height
            xbr = float(image[box].attrib["xbr"])/image_width # x bottom right
            ybr = float(image[box].attrib["ybr"])/image_height
            txt_file.write(f"0 {(xtl+xbr)/2} {(ytl+ybr)/2} {xbr-xtl} {ybr-ytl}\n") # class_id x_center y_center width height
        txt_file.close()

parse_annotation(annot_folder)

### Trénování modelu

In [ ]:
results = model.train(data="dataset.yaml", epochs=100, lr0=0.001)

### Nasazení modelu na test data

In [18]:
trained_model_path = "runs/detect/train_lr0_0001/weights/best.pt"
trained_model = YOLO(trained_model_path)

test_images = []
test_images_path = "datasets/strawberries/test"
for img in os.listdir(test_images_path):
    test_images.append(os.path.join(test_images_path, img))

test_results = trained_model.predict(test_images)

C:\Users\jaros\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\PIL\TiffImagePlugin.py:935: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


0: 640x640 6 strawberrys, 103.1ms
1: 640x640 13 strawberrys, 103.1ms
2: 640x640 5 strawberrys, 103.1ms
3: 640x640 7 strawberrys, 103.1ms
4: 640x640 11 strawberrys, 103.1ms
Speed: 3.1ms preprocess, 103.1ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)


### Zpracování výsledků

In [19]:
imgs_with_boxes = []
for result in test_results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    confidence = boxes.conf.tolist()
    #result.show()  # display to screen

    # DRAW BOXES
    orig_img = cv2.cvtColor(result.orig_img, cv2.COLOR_BGR2RGB)
    boxes_xyxy = boxes.xyxy.tolist()
    for i in range(len(boxes_xyxy)):
        tl = (int(boxes_xyxy[i][0]), int(boxes_xyxy[i][1]))
        br = (int(boxes_xyxy[i][2]), int(boxes_xyxy[i][3]))
        tl_for_text = (int(boxes_xyxy[i][0]), int(boxes_xyxy[i][1] + 0.3*(int(boxes_xyxy[i][3]) - int(boxes_xyxy[i][1])))) # Posunout o 30 % níž
        orig_img = cv2.rectangle(orig_img, tl, br, (255, 0, 0), 2)
        orig_img = cv2.putText(orig_img, str(round(confidence[i],  2)), tl_for_text, 5, result.orig_shape[1]/550, (255, 255, 0), 2)
    imgs_with_boxes.append(orig_img)

# SHOW IMAGES WITH BOXES
plt.figure(figsize=(8, 30))  
for i in range(len(imgs_with_boxes)):
    plt.subplot(5, 1, i+1)
    plt.imshow(imgs_with_boxes[i])
plt.show()

<Figure size 800x3000 with 5 Axes>

In [20]:
print(model.val())

Ultralytics 8.3.51  Python-3.11.9 torch-2.5.1+cpu CPU (Intel Core(TM) i5-7600K 3.80GHz)


val: Scanning E:\Skola\TUL-navazující_studium\3. semestr\PVI-Pocitacove_videni\cviceni\cv13\datasets\strawberries\labels.cache... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.15s/it]


                   all         40        475      0.955      0.931      0.981       0.81
Speed: 1.4ms preprocess, 64.0ms inference, 0.0ms loss, 9.3ms postprocess per image
Results saved to runs\detect\train3
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002A286D16BD0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    